In [ ]:
import math
# !pip install deepeval
# !pip install langchain_community
# !pip install -qU pypdf
# !pip install gigachain-community gigachain-chroma
# !pip install sentence-transformers
# !pip install langsmith
# !pip install pymupdf

import nltk

nltk.download('punkt')
from langchain_chroma import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

import os
import re
from langchain_community.document_loaders import PyPDFLoader

In [7]:
# from config import sber
sber = "NjM5NzdhZmQtZDg0Yy00NmVjLTgyYTctNTViMjYwMGRjMmQ3OjdkMmM0MmQ0LWZkNzItNDczNy1iNjliLTlkYjU2MjE1NWFlYw=="
embedding_function_gigachat = GigaChatEmbeddings(credentials=sber, verify_ssl_certs=False,
                                                 scope="GIGACHAT_API_PERS")

In [ ]:
cb_rf_acts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_acts/'
cb_rf_info_letters = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_info_letters/'
cb_rf_methodical_recommendations = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_methodical_recommendations/'
cb_rf_policies = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_policies/'
cb_rf_position = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_position/'
cb_rf_prikazy = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_prikazy/'
cb_rf_recommendations = '/Users/gd/PycharmProjects/altirix_systems_chatbot/cb_rf_recommendations/'
fsb_documents_pdf = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fsb_documents_pdf/'
fstek_documents_pdf = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fstek_documents_pdf/'
fz = '/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/'
gossopka_acts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_acts/'
gosts = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gosts/'
minimum_docs = '/Users/gd/PycharmProjects/altirix_systems_chatbot/minimum_docs/'
# kaspersky_encyclopedia = '/Users/gd/PycharmProjects/altirix_systems_chatbot/kaspersky_encyclopedia/'
# gossopka_glossary = '/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_glossary/'

In [ ]:
docs_path = []

In [ ]:
for path in sorted(os.listdir(gosts)):
    docs_path.append(gosts + path)

In [ ]:
len(docs_path)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def clean_text(file_path):
    docs = PyPDFLoader(file_path).load()
    for i in range(len(docs)):
        if i == 0:
            docs[i].page_content = re.sub(r'Документ предоставлен.*\n', '', docs[i].page_content, flags=re.DOTALL)
        else:
            docs[i].page_content = re.sub(r'КонсультантПлюс.*\.\.\.', '', docs[i].page_content, flags=re.DOTALL)

    return docs


all_docs = []
for path in docs_path:
    cleaned_docs = clean_text(path)
    all_docs.append(cleaned_docs)


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",
        "\uff0c",
        "\u3001",
        "\uff0e",
        "\u3002",
        "\t",
        ''
    ],
    chunk_size=1200,
    chunk_overlap=70,
    length_function=len
)

In [ ]:
for i in range(len(all_docs)):
    for j in range(len(all_docs[i])):
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\xad\n', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace(' -\n', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("-\n", "")
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("\n", " ")
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\u2002', ' ')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\xa0', ' ')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace('\\u', '')
        all_docs[i][j].page_content = all_docs[i][j].page_content.replace("\t", " ")

In [ ]:
chunks = []
for docs in all_docs:
    chunks.append(text_splitter.split_documents(docs))

In [ ]:
import chromadb

persist_directory = "/Users/gd/PycharmProjects/altirix_systems_chatbot/chromadb_chunk_size_1200"
client = chromadb.PersistentClient(path=persist_directory)
client.get_or_create_collection(name="chromadb_chunk_size_1200")

In [8]:
chromadb_chunk_size_1200 = Chroma(
    persist_directory="/Users/gd/PycharmProjects/altirix_systems_chatbot/chromadb_chunk_size_1200",
    embedding_function=embedding_function_gigachat)


In [9]:
db_info = chromadb_chunk_size_1200._collection.get(include=["documents", "metadatas", "embeddings"])

In [ ]:
for i in range(len(db_info['metadatas'])):
    if 'Указ Президента РФ от 30.03.2022 N 166 (ред. от 22.11.2023).pdf' in db_info['metadatas'][i]['source']:
        print('_______')
        print(db_info['documents'][i])

In [ ]:
ids_ = []
metadatas_ = []
documents_ = []
embs_ = []
for i in range(len(db_info['metadatas'])):
    if 'gosts' not in db_info['metadatas'][i]['source']:
        ids_.append(db_info['ids'][i])
        documents_.append(db_info['documents'][i])
        embs_.append(db_info['embeddings'][i])
        metadatas_.append(db_info['metadatas'][i])



In [ ]:
documents_strs = []
metadatas = []
for i in range(len(chunks)):
    for d in chunks[i]:
        documents_strs.append(d.page_content)
        metadatas.append(d.metadata)

ids = [f'doc{i}' for i in range(len(db_info['ids']), len(metadatas) + len(db_info['ids']))]

In [ ]:
len(ids)

In [ ]:
import time
from gigachat.exceptions import ResponseError

embedded_docs = []
doc_idx_cant_process = []
for i in range(len(documents_strs)):
    try:
        emb = embedding_function_gigachat.embed_documents([documents_strs[i]])
        embedded_docs.append(emb)
    except ResponseError:
        doc_idx_cant_process.append(i)
        embedded_docs.append(-1)

    # if i % 10 == 0:
    #     time.sleep(1)


In [ ]:
valid_docs = []
valid_embs = []
valid_metadatas = []
for i in range(len(embedded_docs)):
    if embedded_docs[i] == -1:
        pass
    else:
        valid_docs.append(documents_strs[i])
        valid_embs.append(embedded_docs[i])
        valid_metadatas.append(metadatas[i])

ids = [f'doc{i}' for i in range(len(db_info['ids']) - len(doc_idx_cant_process),
                                len(valid_docs) + len(db_info['ids']) - len(doc_idx_cant_process))]

In [ ]:
valid_embs_ = []
for emb in valid_embs:
    valid_embs_.append(emb[0])

In [ ]:
len(valid_docs), len(valid_embs_), len(valid_metadatas), len(ids)

In [ ]:
#Возможно за раз не нужно добавлять сразу все документы, лучше (по частям)
chromadb_chunk_size_1200._collection.add(
    documents=valid_docs,
    embeddings=valid_embs_,
    ids=ids,
    metadatas=valid_metadatas
)

In [3]:
sources = []
for i in db_info['metadatas']:
    sources.append(i['source'])


NameError: name 'db_info' is not defined

In [47]:
gosts = []
cnt_ = 0
for p in list(set(sources)):
    if 'FZ' in p:
        print(p)



/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/Ob_elektronnoj_podpisi_ot_06.04.2011_N_63_FZ_red._ot_14.07.2022_.pdf
/Users/gd/PycharmProjects/altirix_systems_chatbot/fz/Federal_nyy_zakon_ot_26.07.2017_N_187_FZ_O_bezopasnosti_kri.pdf
/Users/gd/PycharmProjects/altirix_systems_chatbot/gossopka_acts/194-FZ.pdf


In [43]:
cnt_

165